In [1]:
#from src.conf.NWPaths import NWPaths
from conf import catalog, globals
from pyspark.sql import SparkSession

from collaborative.pipelines.pipelines import Pipelines
from dotenv import load_dotenv
import mlflow
import os
import logging

In [8]:
client = mlflow.tracking.MlflowClient()
model_version = client.get_latest_versions(
    name=globals.MLflow.ALS_REGISTERED_MODEL_NAME, stages=["None"]
)[0]

In [10]:
model_version
client.transition_model_version_stage(
        name=globals.MLflow.ALS_REGISTERED_MODEL_NAME,
        version=model_version,
        stage="Staging",
        archive_existing_versions=True,
    )


RestException: BAD_REQUEST: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type integer: "<ModelVersion: aliases=[], creation_timestamp=1685748519572, current_stage='None', description='', last_updated_timestamp=1685748519572, name='spark_als_model', run_id='cd7f6c409534451291b05ecb435bef05', run_link='', source='s3://mlflow/1/cd7f6c409534451291b05ecb435bef05/artifacts/als_model', status='READY', status_message='', tags={}, user_id='', version='36'>"
LINE 3: ...= 'spark_als_model' AND model_versions.version != '<ModelVer...
                                                             ^

[SQL: SELECT model_versions.name AS model_versions_name, model_versions.version AS model_versions_version, model_versions.creation_time AS model_versions_creation_time, model_versions.last_updated_time AS model_versions_last_updated_time, model_versions.description AS model_versions_description, model_versions.user_id AS model_versions_user_id, model_versions.current_stage AS model

In [2]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'

In [12]:
logging.getLogger("mlflow").setLevel(logging.DEBUG)
logging.getLogger("pyspark").setLevel(logging.DEBUG)

In [13]:
spark = (
    SparkSession.builder.appName("Next Watch ML")
    .master("local[3]")
    # .config("spark.executor.memory", "3g")
    .config("spark.driver.maxResultSize", "96g")
    .config("spark.driver.memory", "96g")
    .config("spark.executor.memory", "8g")
    .config("packages", "org.apache.hadoop:hadoop-aws:3.3.4")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

In [14]:
""" Hadoop version"""
spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.3.4'

In [6]:
load_dotenv("../.env")

True

In [16]:
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
mlflow.set_experiment(globals.MLflow.EXPERIMENT_NAME)

<Experiment: artifact_location='s3://mlflow/1', creation_time=1685238316335, experiment_id='1', last_update_time=1685238316335, lifecycle_stage='active', name='COLLABORATIVE', tags={}>

In [17]:
mlflow.spark.autolog(disable=True)

2023/05/28 18:05:12 DEBUG mlflow.utils.autologging_utils: Called autolog() method for spark autologging with args '()' and kwargs '{'disable': True, 'silent': False}'


In [18]:
pipelines = Pipelines(spark, catalog.Sources.MOVIELENS)

In [19]:
train, serve = pipelines.data_engineering()

In [10]:
#train.count()

In [9]:
pipelines.data_science() #train

  0%|                           | 0/5 [00:00<?, ?trial/s, best loss=?]

23/05/28 16:47:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/28 16:47:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
/home/bruno/conda/envs/next-watch/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



  0%|                           | 0/5 [00:54<?, ?trial/s, best loss=?]


KeyboardInterrupt: 